# Shopify Challenge 2021
## Data Science Internship 
## Jose Rincon

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Question 1:
#### On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of  3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

### a. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
### Solution a
#### Read data using pandas

In [2]:
my_data = pd.read_excel("/home/jose/Documents/Professional Development/shopify_challenge/2019 Winter Data Science Intern Challenge Data Set.xlsx", engine = 'openpyxl')


#### Perform 30 day window average using the original computation 

In [3]:
order_values = my_data['order_amount'].to_numpy()
store_ids = my_data['shop_id'].to_numpy()
average_order_value = np.mean(order_values)

In [4]:
print(average_order_value)

3145.128


#### Find any possible outliers in the dataset.  We do this by finding the average and standard deviation of number of items in an order. Orders that fall within three standard deviations could be deemed common for the stores. Orders with very large number of shoes could be unusual for the stores. Those outliers could be ignored in our calculation.  

In [5]:
total_items = my_data['total_items'].to_numpy()

#### Compute mean and standard deviation of total_items per order

In [6]:
mean = np.mean(total_items)
std = np.std(total_items)
median = np.median(total_items)

In [7]:
print(mean, std, median)

8.7872 116.3086871912842 2.0


#### Find outliers in store orders 

In [8]:
# Use 3 standard deviations to find ourliers
cut_off = 3 * std 
# find lower boundary of our good data
lower = mean - cut_off 
# find upper boundary of our good data
upper = mean + cut_off
# find the indices of outliers
indices_outliers =  (total_items < lower) + (total_items > upper)
# find the number of items in outliers
outliers_total_items = total_items[indices_outliers]
print(outliers_total_items)
# find the store ids with the outliers
outliers_store_ids = store_ids[indices_outliers]
print(outliers_store_ids)


[2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000
 2000 2000 2000]
[42 42 42 42 42 42 42 42 42 42 42 42 42 42 42 42 42]


#### They are orders with 2000 items sold by store 42. 
#### Let's now remove the outliers from our data

In [9]:
# get new_total_items 
new_total_items = total_items[~indices_outliers]
# get new_order_values 
new_order_values = order_values[~indices_outliers]
# get max, min, mean, std of  new_total_items 
max_total_items = np.max(new_total_items)
min_total_items = np.min(new_total_items)
average_total_items = np.average(new_total_items)
std_total_items = np.std(new_total_items)
print('max_total_items', max_total_items)
print('min_total_items', min_total_items)
print('average_total_items', average_total_items)
print('std_total_items', std_total_items)



max_total_items 8
min_total_items 1
average_total_items 1.9939795304033714
std_total_items 0.9830817903534801


#### There are 17 outliers from our 5000 order data set. I am not sure why store 42 has those 2000 item orders made by their customer 607 but these type of orders are not usual for a retail shoe store. Moreover, there are also outliers in store  78 where shoes are sold for a unit price of 25725.  These outliers are definetely affecting the Average Order Value (AOV). 

### b. What metric would you report for this dataset?
### Solution b

#### Given that this data set has a few very large outliers, the mean is skewed by these few samples. We could use the median as better value of central tendency.  

### c. What is the value?
### Solution: The Median Order Value is


In [13]:
median_order_value = np.median(order_values)

In [14]:
print("The Median Order Value (MOV) is: ", median_order_value)

The Median Order Value (MOV) is:  284.0


## Question 2:
#### For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below

### a. How many orders were shipped by Speedy Express in total?


In [20]:
'''
SELECT COUNT(tempOrders.ShipperID)
FROM Orders AS tempOrders
WHERE (SELECT ShipperID 
    FROM Shippers AS tempShippers
    WHERE tempShippers.ShipperName == "Speedy Express") == tempOrders.ShipperID'''

'\nSELECT COUNT(tempOrders.ShipperID)\nFROM Orders AS tempOrders\nWHERE (SELECT ShipperID \n    FROM Shippers AS tempShippers\n    WHERE tempShippers.ShipperName == "Speedy Express") == tempOrders.ShipperID'

### Solution a: The number of orders shipped by Speedy Express were 54

### b. What is the last name of the employee with the most orders?


In [23]:
'''
SELECT Employees.LastName
FROM Employees
LEFT JOIN Orders
ON Orders.EmployeeID = Employees.EmployeeID
GROUP BY Orders.EmployeeID
ORDER BY COUNT(Orders.EmployeeID) DESC
LIMIT 1;'''

'\nSELECT Employees.LastName\nFROM Employees\nLEFT JOIN Orders\nON Orders.EmployeeID = Employees.EmployeeID\nGROUP BY Orders.EmployeeID\nORDER BY COUNT(Orders.EmployeeID) DESC\nLIMIT 1;'

### Solution b: The employee with most orders is Peacock

### c. What product was ordered the most by customers in Germany?



In [24]:
'''SELECT 
Products.ProductName, SUM(OrderDetails.Quantity) AS Total_orders, Customers.Country
FROM Products
JOIN OrderDetails ON OrderDetails.ProductID = Products.ProductID
JOIN Orders ON Orders.OrderID = OrderDetails.OrderID
JOIN Customers ON Customers.CustomerID = Orders.CustomerID
WHERE Customers.Country = "Germany"
GROUP BY Products.ProductName
ORDER BY Total_orders DESC
LIMIT 1;'''

'SELECT \nProducts.ProductName, SUM(OrderDetails.Quantity) AS Total_orders, Customers.Country\nFROM Products\nJOIN OrderDetails ON OrderDetails.ProductID = Products.ProductID\nJOIN Orders ON Orders.OrderID = OrderDetails.OrderID\nJOIN Customers ON Customers.CustomerID = Orders.CustomerID\nWHERE Customers.Country = "Germany"\nGROUP BY Products.ProductName\nORDER BY Total_orders DESC\nLIMIT 1;'

### Solution c. Boston Crab Meat was the most ordered product in Germany